In [1]:
print("a")

a


In [4]:
%pip install datasets

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.9/30.9 MB 1.5 MB/s eta 0:00:0000:0100:01
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: requests
    Found existing installation: requests 2.22.0
    Uninstalling requests-2.22.0:
      Successfully uninstalled requests-2.22.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 12.0.1
    Uninstalling pyarrow-12.0.1:
      Successfully uninstalled pyarrow-12.0.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━ 12/16 [huggingface-hub]  WARNING: The script huggingface-cli is installed in '/Library/Frameworks/Python.framework/Versions/3.11/bin' which is not on PATH.
  Consider adding this directory to PATH or

In [3]:
from datasets import load_dataset
try:
    train_ds = load_dataset("grammarly/coedit", split="train", cache_dir="./data")
    val_ds = load_dataset("grammarly/coedit", split="validation", cache_dir="./data")
except ValueError as e:
    print(f"ran into error while loading dataset: {e}")



In [4]:
print(train_ds)
print(val_ds)

Dataset({
    features: ['_id', 'task', 'src', 'tgt'],
    num_rows: 69071
})
Dataset({
    features: ['_id', 'task', 'src', 'tgt'],
    num_rows: 1712
})


In [31]:
%pip uninstall google-genai -y
%pip install -U -q "google-genai>=1.10.0"

Found existing installation: google-genai 1.13.0
Uninstalling google-genai-1.13.0:
  Successfully uninstalled google-genai-1.13.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [77]:
from google import genai
from google.genai import types
from dotenv import load_dotenv
import os

load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")
client = genai.Client(api_key=gemini_api_key)
MODEL_ID="gemini-2.5-flash-preview-04-17"


src = "Remove all grammatical errors from this text: Safety is one of the crucial problems that many countries and companies concern."
tgt = "Safety is one of the crucial problems that many countries and companies are concerned about."
prompt = f"Source (src): {src}\nTarget (tgt): {tgt}"

sys_prompt = """
```system
You are an expert linguistic analyzer simulating the process of text editing. You will receive a source text (`src`) which contains an editing instruction followed by the text to be edited.

Your task is to:
1.  Carefully identify the editing instruction given at the beginning of the `src` text.
2.  Analyze the original text segment that follows the instruction in `src`.
3.  Apply the instruction step-by-step to the original text.
4.  For each step or change you make, articulate the precise reasoning based on linguistic rules, the instruction's goal (e.g., grammar, clarity, simplification), or common writing conventions. Explain *why* a change is necessary or beneficial according to the instruction.
5.  Your reasoning should demonstrate *how* to arrive at the corrected text, simulating the thought process of performing the edit yourself.

**Crucially:** Your reasoning process should *not* refer to or imply knowledge of a pre-existing target or corrected text. Focus *only* on applying the instruction to the source text to derive the necessary changes.

Your entire output must consist *only* of the step-by-step reasoning process, enclosed within `<think></think>` tags. Do not include the original instruction, the source text, the final corrected text, or any other text outside these tags.

**Example Interaction:**

**User Input (constructed from your dataset):**

```
Source (src): Fix grammar in this sentence: If engineers do not come up with new ideas, they cannot find best solution for the problems.
Target (tgt): If engineers do not come up with new ideas, they cannot find the best solution for different problems.
```

**Your Expected Output (demonstrating the reasoning to *derive* the correction):**

```xml
<think>
1. Instruction analysis: The task is to "Fix grammar". This requires identifying and correcting grammatical errors in the sentence "If engineers do not come up with new ideas, they cannot find best solution for the problems."
2. Sentence segment analysis: The first clause "If engineers do not come up with new ideas" appears grammatically sound.
3. Focus on the second clause: "they cannot find best solution for the problems."
4. Identify potential error 1: The noun phrase "best solution". "Best" is a superlative adjective. Grammatical rule: Superlative adjectives modifying singular countable nouns generally require a definite article ("the").
5. Apply correction 1: Insert "the" before "best solution". The phrase becomes "the best solution".
6. Identify potential error/improvement area 2: The phrase "for the problems". While grammatically plausible, using "the" implies specific, previously identified problems. The context seems general. Improving grammatical flow or clarity might involve adjusting this.
7. Consider alternatives for "the problems": "for problems" (too general?), "for specific problems" (adds info), "for different problems" (suggests variety, often fits general statements well).
8. Apply correction 2: Replace "the problems" with "different problems" for better contextual fit and naturalness, aligning with the goal of general grammatical improvement and clarity often included in "Fix grammar" tasks.
9. Synthesized correction based on reasoning: The corrected clause is "they cannot find the best solution for different problems." The full sentence derived from applying the grammar fixes is "If engineers do not come up with new ideas, they cannot find the best solution for different problems."
</think>
```
"""




response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=types.GenerateContentConfig(
        system_instruction=sys_prompt,
        thinking_config=types.ThinkingConfig(
            thinking_budget=500 # 0 means no thinking None means no limit to thinking it can take however many tokens it wants.
        )
    )
)


print(response.text)
print()
print("Prompt tokens:",response.usage_metadata.prompt_token_count)
print("Thoughts tokens:",response.usage_metadata.thoughts_token_count)
print("Output tokens:",response.usage_metadata.candidates_token_count)
print("Total tokens:",response.usage_metadata.total_token_count)

<think>
1.  **Instruction Analysis:** The task is to "Remove all grammatical errors" from the provided sentence. This requires identifying and correcting any grammatical mistakes in the sentence "Safety is one of the crucial problems that many countries and companies concern."
2.  **Sentence Analysis:** The sentence structure is "Subject (Safety) + Verb (is) + Complement (one of the crucial problems) + Relative Clause (that many countries and companies concern)".
3.  **Focus on the Relative Clause:** The phrase "that many countries and companies concern" is where the potential error lies. The verb "concern" is being used actively here, apparently with "many countries and companies" as the subject and "problems" (implied as the antecedent of "that") as the direct object.
4.  **Analyze the Usage of "concern":** The active verb "concern" typically means 'to relate to' or 'to affect'. For example, "The rule concerns all employees" or "This matter concerns your safety." In the source senten

In [78]:
# Pricing per 1 Million tokens
INPUT_PRICE_PER_MILLION = 0.15
OUTPUT_NON_THINKING_PRICE_PER_MILLION = 0.60
OUTPUT_THINKING_PRICE_PER_MILLION = 3.50

# --- Get Token Counts from User ---
print("Enter the token counts (use 0 if Thoughts tokens is None or zero):")

prompt_tokens = 800
thoughts_tokens = 393 # Assume 0 if None was the actual value
output_tokens = 609

# --- Calculate Costs ---

# Input Cost
input_cost = (prompt_tokens / 1_000_000) * INPUT_PRICE_PER_MILLION

# Output Cost (depends on whether thinking tokens were generated)
if thoughts_tokens > 0:
    # Thinking was used - price applies to output + thoughts tokens
    billable_output_tokens = output_tokens + thoughts_tokens
    output_cost = (billable_output_tokens / 1_000_000) * OUTPUT_THINKING_PRICE_PER_MILLION
    print("\nCalculating using THINKING output price ($3.50 / 1M tokens)")
else:
    # No thinking - price applies only to output tokens
    billable_output_tokens = output_tokens
    output_cost = (billable_output_tokens / 1_000_000) * OUTPUT_NON_THINKING_PRICE_PER_MILLION
    print("\nCalculating using NON-THINKING output price ($0.60 / 1M tokens)")

# Total Cost
total_cost = input_cost + output_cost

# --- Display Results ---
print(f"\n--- Cost Breakdown ---")
print(f"Input Cost:       ${input_cost:.6f}")
print(f"Output Cost:      ${output_cost:.6f} (based on {billable_output_tokens} billable output tokens)")
print(f"----------------------")
print(f"Total Estimated Cost: ${total_cost:.6f}")

# Example calculation for 10,000 identical calls
num_calls = 10000
total_cost_10k = total_cost * num_calls
print(f"\nEstimated Cost for {num_calls:,} calls: ${total_cost_10k:.2f}")

Enter the token counts (use 0 if Thoughts tokens is None or zero):

Calculating using THINKING output price ($3.50 / 1M tokens)

--- Cost Breakdown ---
Input Cost:       $0.000120
Output Cost:      $0.003507 (based on 1002 billable output tokens)
----------------------
Total Estimated Cost: $0.003627

Estimated Cost for 10,000 calls: $36.27
